In [1]:
from dataloader import GraphTextDataset, GraphDataset, TextDataset, AddRWStructEncoding
from torch_geometric.loader import DataLoader
from torch.utils.data import DataLoader as TorchDataLoader
from Model import Model
import numpy as np
from transformers import AutoTokenizer
import torch
from torch import optim
import time
import os
import pandas as pd

/home/cedric/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CE = torch.nn.CrossEntropyLoss()
def contrastive_loss(v1, v2):
  logits = torch.matmul(v1,torch.transpose(v2, 0, 1))
  labels = torch.arange(logits.shape[0], device=v1.device)
  return CE(logits, labels) + CE(torch.transpose(logits, 0, 1), labels)

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

graph_model_name = 'gps'

gt = np.load("./data/token_embedding_dict.npy", allow_pickle=True)[()]
walk_length = 20
val_dataset = GraphTextDataset(root='./data/', gt=gt, split='val', tokenizer=tokenizer, graph_transform=AddRWStructEncoding(walk_length))
train_dataset = GraphTextDataset(root='./data/', gt=gt, split='train', tokenizer=tokenizer, graph_transform=AddRWStructEncoding(walk_length))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nb_epochs = 5
batch_size_train = 16
batch_size_test = 16
learning_rate = 2e-5
graph_config = {}
graph_config['graph_layer'] = 10
graph_config['n_head'] = 4
graph_config['n_feedforward'] = 128
graph_config['input_dropout'] = 0.1
graph_config['dropout'] = 0.0
graph_config['attention_dropout'] = 0.25
graph_config['conv_type'] = 'Gated'
graph_config['walk_length'] = 20
graph_config['dim_se'] = 28

val_loader = DataLoader(val_dataset, batch_size=batch_size_test, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

model = Model(model_name=model_name, graph_model_name=graph_model_name, num_node_features=300, nout=768, nhid=300, graph_hidden_channels=64, graph_config=graph_config) # nout = bert model hidden dim
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate,
                                betas=(0.9, 0.999),
                                weight_decay=0.01)

epoch = 0
loss = 0
losses = []
count_iter = 0
time1 = time.time()
printEvery = 50
best_validation_loss = 1000000

Processing...
/media/cedric/Stockage1/Documents/Cours/MVA/Semestre1/ALTEGRAD/Challenge/Data-Challenge-Molecules/dataloader.py:94: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(edge_index).T, torch.FloatTensor(x)
/media/cedric/Stockage1/Documents/Cours/MVA/Semestre1/ALTEGRAD/Challenge/Data-Challenge-Molecules/dataloader.py:94: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  return torch.LongTensor(edge_index).T, torch.FloatTensor(x)
Done!
Proc

In [3]:
total_params = sum(p.numel() for p in model.parameters())
graph_params = sum(p.numel() for p in model.graph_encoder.parameters())
text_params = sum(p.numel() for p in model.text_encoder.parameters())

print(f'Total number of parameters: {total_params:,}')
print(f'    Graph encoder: {graph_params:,} parameters')
print(f'    Text encoder: {text_params:,} parameters')

Total number of parameters: 67,127,412
    Graph encoder: 764,532 parameters
    Text encoder: 66,362,880 parameters


### Train

In [4]:
model_save_name = f'{model_name}__{graph_model_name}_{3}_{300}_{graph_params//1000}m__base_'
model_save_name

'distilbert-base-uncased__gps_3_300_764m__base_'

In [5]:
for i in range(nb_epochs):
    print('-----EPOCH {}-----'.format(i+1))
    model.train()
    for batch in train_loader:
        input_ids = batch.input_ids
        batch.pop('input_ids')
        attention_mask = batch.attention_mask
        batch.pop('attention_mask')
        graph_batch = batch
        
        x_graph, x_text = model(graph_batch.to(device), 
                                input_ids.to(device), 
                                attention_mask.to(device))
        current_loss = contrastive_loss(x_graph, x_text)   
        optimizer.zero_grad()
        current_loss.backward()
        optimizer.step()
        loss += current_loss.item()
        
        count_iter += 1
        if count_iter % printEvery == 0:
            time2 = time.time()
            print("Iteration: {0}, Time: {1:.4f} s, training loss: {2:.4f}".format(count_iter,
                                                                        time2 - time1, loss/printEvery))
            losses.append(loss)
            loss = 0 

    model.eval()       
    val_loss = 0        
    for batch in val_loader:
        input_ids = batch.input_ids
        batch.pop('input_ids')
        attention_mask = batch.attention_mask
        batch.pop('attention_mask')
        graph_batch = batch
        x_graph, x_text = model(graph_batch.to(device), 
                                input_ids.to(device), 
                                attention_mask.to(device))
        current_loss = contrastive_loss(x_graph, x_text)   
        val_loss += current_loss.item()
    best_validation_loss = min(best_validation_loss, val_loss)

    print('-----EPOCH '+str(i+1)+'----- done.  Validation loss: ', str(val_loss/len(val_loader)) )
    if best_validation_loss==val_loss:
        print('validation loss improved saving checkpoint...')
        save_path = os.path.join('./checkpoints', model_save_name+str(i)+'.pt')
        torch.save({
        'epoch': i,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'validation_accuracy': val_loss,
        'loss': loss,
        }, save_path)
        print('checkpoint saved to: {}'.format(save_path))

-----EPOCH 1-----
torch.Size([546, 36])
torch.Size([546, 28])
torch.Size([426, 36])
torch.Size([426, 28])
torch.Size([461, 36])
torch.Size([461, 28])
torch.Size([431, 36])
torch.Size([431, 28])
torch.Size([413, 36])
torch.Size([413, 28])
torch.Size([511, 36])
torch.Size([511, 28])
torch.Size([448, 36])
torch.Size([448, 28])
torch.Size([465, 36])
torch.Size([465, 28])
torch.Size([673, 36])
torch.Size([673, 28])
torch.Size([455, 36])
torch.Size([455, 28])
torch.Size([628, 36])
torch.Size([628, 28])
torch.Size([483, 36])
torch.Size([483, 28])
torch.Size([483, 36])
torch.Size([483, 28])
torch.Size([380, 36])
torch.Size([380, 28])
torch.Size([488, 36])
torch.Size([488, 28])
torch.Size([570, 36])
torch.Size([570, 28])
torch.Size([476, 36])
torch.Size([476, 28])
torch.Size([544, 36])
torch.Size([544, 28])
torch.Size([539, 36])
torch.Size([539, 28])
torch.Size([476, 36])
torch.Size([476, 28])
torch.Size([482, 36])
torch.Size([482, 28])
torch.Size([603, 36])
torch.Size([603, 28])
torch.Size([46

KeyboardInterrupt: 

### Submission

In [4]:
#save_path = os.path.join('./checkpoints', 'model'+str(4)+'.pt')

print('loading best model...')
checkpoint = torch.load(save_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

graph_model = model.get_graph_encoder()
text_model = model.get_text_encoder()

test_cids_dataset = GraphDataset(root='./data/', gt=gt, split='test_cids')
test_text_dataset = TextDataset(file_path='./data/test_text.txt', tokenizer=tokenizer)

idx_to_cid = test_cids_dataset.get_idx_to_cid()

test_loader = DataLoader(test_cids_dataset, batch_size=batch_size_test, shuffle=False)

graph_embeddings = []
for batch in test_loader:
    for output in graph_model(batch.to(device)):
        graph_embeddings.append(output.tolist())

test_text_loader = TorchDataLoader(test_text_dataset, batch_size=batch_size_test, shuffle=False)
text_embeddings = []
for batch in test_text_loader:
    for output in text_model(batch['input_ids'].to(device), 
                             attention_mask=batch['attention_mask'].to(device)):
        text_embeddings.append(output.tolist())


from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(text_embeddings, graph_embeddings)

solution = pd.DataFrame(similarity)
solution['ID'] = solution.index
solution = solution[['ID'] + [col for col in solution.columns if col!='ID']]
solution.to_csv('submission.csv', index=False)

loading best model...


### Eval

In [5]:
for i in range(nb_epochs):
    save_path = os.path.join('./checkpoints', 'model'+str(i)+'.pt')
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()       
    val_loss = 0        
    for batch in val_loader:
        input_ids = batch.input_ids
        batch.pop('input_ids')
        attention_mask = batch.attention_mask
        batch.pop('attention_mask')
        graph_batch = batch
        x_graph, x_text = model(graph_batch.to(device), 
                                input_ids.to(device), 
                                attention_mask.to(device))
        current_loss = contrastive_loss(x_graph, x_text)   
        val_loss += current_loss.item()

    print('-----EPOCH '+str(i+1)+'----- done.  Validation loss: ', str(val_loss/len(val_loader)) )

-----EPOCH 1----- done.  Validation loss:  0.9059125973262648
-----EPOCH 2----- done.  Validation loss:  0.5076332101664112
-----EPOCH 3----- done.  Validation loss:  0.3639905517425946
-----EPOCH 4----- done.  Validation loss:  0.32216463984858584
-----EPOCH 5----- done.  Validation loss:  0.29098984794151306
